# 计数器

香山实现了全面的性能计数器，分为以下三种：

- Accumulation：基础的累加型计数器（log 打印）
- Histogram：统计数值分布（log 打印）
- Rolling：采集片段计数以分析性能变化（ChiselDB 存储）

Accumulation 类型的性能计数器在仿真结束后打印到 stderr。在此之前的每次执行都会输出这些数据，只不过我们都将他们丢弃了。现在，我们来看一看这些数据。由于这些数据太大了，因此我们只展示最后的 100 条

In [ ]:
%%bash
cd ../ && source env.sh >/dev/null
cd ${NOOP_HOME}

${READY2RUN_HOME}/emu \
    -i ${READY2RUN_HOME}/hello-riscv64-xs.bin \
    --no-diff 2>stderr.log

echo "=== Last 10 lines:"
tail -n 10 stderr.log

echo "=== Example of XSPerfAccumulate: rob commitInstr"
grep -n "rob: commitInstr," stderr.log

echo "=== Example of XSHistogram: l2cache acquire period"
grep -n "l2cache.slices_0.mshrCtl: acquire_period" stderr.log

在 [https://github.com/OpenXiangShan/env-scripts/blob/main/perf/perf.py](https://github.com/OpenXiangShan/env-scripts/blob/main/perf/perf.py) 可以找到更多的数据分析脚本

Rolling 采用滑动窗口的思想，采集程序运行过程中每个片段（i.e. 每 10000 周期）的性能数据，用于分析程序行为特征和变化。

要启用 RollingDB，需要在编译时指定 `WITH_ROLLINGDB=1`，并在运行时指定 `--dump-db` 参数。本次 tutorial 中受时间和运算资源限制，请不要运行编译和运行的代码。

我们提供了用此方式采集得到的 `xs-perf-rolling.db` 文件，您可以使用 `${NOOP_HOME}/scripts/rolling` 下的 python 脚本对其进行分析。您可以尝试运行下面的单元格。

In [ ]:
%%bash
cd ../ && source env.sh >/dev/null

# 编译和运行：注意，请不要运行这部分
# cd ${NOOP_HOME}
# make clean
# make emu EMU_THREADS=4 WITH_CHISELDB=1 WITH_ROLLINGDB=1 -j8 \
#     PGO_WORKLOAD=${NOOP_HOME}/ready-to-run/coremark-2-iteration.bin \
#     PGO_MAX_CYCLE=10000 PGO_EMU_ARGS=--no-diff LLVM_PROFDATA=llvm-profdata
# ./build/emu -i ./ready-to-run/coremark-2-iteration.bin \
#     --diff ./ready-to-run/riscv64-nemu-interpreter-so --dump-db
# cp `find $NOOP_HOME/build/ -type f -name "*.db" | tail -1` \
#     ${XS_PROJECT_ROOT}/tutorial/${dir}/xs-perf-rolling.db

# 使用脚本对 rolling db 进行分析
cd ${NOOP_HOME}/scripts/rolling
python3 rollingplot.py ${READY2RUN_HOME}/xs-perf-rolling.db ipc

该脚本会输出以下图片，可以看到 XiangShan 在运行这一程序时，每一段时间内的 IPC 变化：

![perf](../XiangShan//scripts/rolling/results/perf.png)

# Topdown

香山在 RTL 上实现了完善的 Topdown 计数器，并针对 RISC-V 和香山微架构进行了优化

Topdown 计数器的结果在仿真结束后会输出到 stderr。您可以使用我们准备好的脚本对香山的 Topdown 结果进行分析

由于 SPEC2006 的切片数量非常多，且性能相关的数据量很大，以下脚本会涉及大量文件读写，因此我们不在现场执行这些分析脚本

In [2]:
%%bash
cd ../ && source env.sh >/dev/null

# 请不要执行以下命令！
# cd ${NOOP_HOME}/scripts/top-down && python3 top_down.py -s ${READY2RUN_HOME}/SPEC06_EmuTasks_topdown -j ${READY2RUN_HOME}/SPEC06_EmuTasks_topdown.json
# ls ${NOOP_HOME}/scripts/top-down/results